In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\MEDINA TECH\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MEDINA TECH\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\MEDINA TECH\anaconda3\lib\site-packages\numpy\.libs\libopenblas.qvlo2t66wepi7jz63ps3hmohfey472bc.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
!nvidia-smi

Tue Mar 29 20:08:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.19       Driver Version: 472.19       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P0    N/A /  N/A |     88MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install tensorflow-gpu

You should consider upgrading via the 'c:\users\medina tech\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2

In [5]:
IMAGE_SIZE = [299, 299]
TRAIN_PATH = 'F:/RiceDataset(Segmented)/train'
VALIDATION_PATH = 'F:/RiceDataset(Segmented)/validation',
TEST_IMAGE = 'E:/Official purpose/Sprint - 26/Crop images/ricebrownnarrowspot.jpg'

In [6]:
vgg = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(299, 299, 3)))

In [7]:
vgg.trainable = False

In [8]:
folders = glob('F:/RiceDataset(Segmented)/train/*')

In [9]:
flatten = vgg.output
flatten = Flatten()(flatten)

In [10]:
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(len(folders), activation="softmax")(bboxHead)

In [11]:
model = Model(inputs=vgg.input, outputs=bboxHead)

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0     

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('F:/RiceDataset(Segmented)/train/',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 320 images belonging to 7 classes.


In [15]:
validation_set = test_datagen.flow_from_directory('F:/RiceDataset(Segmented)/validation/',
                                                  target_size = (299, 299),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

Found 80 images belonging to 7 classes.


In [16]:
trained_model = model.fit(
  training_set,
  validation_data = validation_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set)
)

Epoch 1/50
10/10 [==============================] - 145s 15s/step - loss: 3.3567 - accuracy: 0.2101 - val_loss: 1.7994 - val_accuracy: 0.4000
Epoch 2/50
10/10 [==============================] - 340s 35s/step - loss: 1.5593 - accuracy: 0.4417 - val_loss: 1.4909 - val_accuracy: 0.4500
Epoch 3/50
10/10 [==============================] - 337s 33s/step - loss: 1.1529 - accuracy: 0.5978 - val_loss: 1.3465 - val_accuracy: 0.4750
Epoch 4/50
10/10 [==============================] - 306s 31s/step - loss: 0.8713 - accuracy: 0.6792 - val_loss: 1.2531 - val_accuracy: 0.5375
Epoch 5/50
10/10 [==============================] - 333s 34s/step - loss: 0.7109 - accuracy: 0.7262 - val_loss: 1.2571 - val_accuracy: 0.5750
Epoch 6/50
10/10 [==============================] - 357s 36s/step - loss: 0.6961 - accuracy: 0.7261 - val_loss: 1.2449 - val_accuracy: 0.5750
Epoch 7/50
10/10 [==============================] - 337s 34s/step - loss: 0.6766 - accuracy: 0.7353 - val_loss: 1.3726 - val_accuracy: 0.4875
Epoch 

In [17]:
model.save('E:/Official purpose/2022/Sprint06/rice_model(vgg).h5')

In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [20]:
model=load_model('E:/Official purpose/2022/Sprint06/rice_model(vgg).h5')

In [28]:
import cv2, imutils
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('E:/Official purpose/2022/Sprint06/rice_model(vgg).h5')
img=image.load_img('E:/Official purpose/Sprint - 26/Crop images/rice_hispa.jpg',target_size=(299, 299))
img_arr=image.img_to_array(img)
img_arr=img_arr/255
img_arr=np.expand_dims(img_arr,axis=0)
preds = model.predict(img_arr)
print(preds)
(startX, startY, endX, endY) = preds[:, 0], preds[:, 1], preds[:, 2], preds[:, 3]
image = cv2.imread('E:/Official purpose/Sprint - 26/Crop images/rice_hispa.jpg')
image = imutils.resize(image, width=600)
(h, w) = image.shape[:2]
startX = int(startX * w)
startY = int(startY * h)
endX = int(endX * w)
endY = int(endY * h)
cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
cv2.imshow("Output", image)
cv2.waitKey(0)

[[3.2608164e-04 1.8896939e-03 2.8843053e-05 1.4727510e-04 3.4263143e-05
  9.9533647e-01 2.2375043e-03]]


-1